In [ ]:
!pip install fair-esm

In [1]:
import numpy as np
import pandas as pd
import torch
import esm

In [2]:
df_main = pd.read_csv("raw_data/third_merge.csv")

In [ ]:
def mini_embedding(df):
    model, alphabet = esm.pretrained.esm2_t12_35M_UR50D()
    batch_converter = alphabet.get_batch_converter()
    model.eval()  # disables dropout for deterministic results
    Uniprot_id = df['UniProt'].to_list()
    Sequence = df['sequence'].to_list()
    data = []
    for i in range(len(Uniprot_id)):
      data.append([Uniprot_id[i], Sequence[i]])
    batch_labels, batch_strs, batch_tokens = batch_converter(data)
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33], return_contacts=True)
    token_representations = results["representations"][33]
    return token_representations

In [3]:
def batch(df, batch_size=1):
    res = []
    for i in range(len(df)):
        res.append(mini_embedding(df.iloc[i:i+1]))
    return res

20412